In [177]:
#import dependencies 
from numpy.random import seed
from numpy.random import randn
from scipy import stats
import numpy as np

import math  

import seaborn as sns
sns.set(style="darkgrid")
from matplotlib import pyplot as plt

import pandas as pd
import os

In [178]:
#check list of files in data directory 
data_folder = './data/dedicated_host/c4/'
print(os.listdir(data_folder))

['c4_ycruncher_dedicated_host_no_idle_-s_flag_11-25-2019_18_07~18_28-us-east-1b - raw.csv', 'c4_memory_sysbench_dedcated_host_no_idle - raw.csv', 'c4_sysbench_dedicated_host_no_idle_11-25-2019_18_37~18_58_us-east-1b - raw.csv']


In [264]:
#make sure to import correct file here
data_name = 'c4_memory_sysbench_dedcated_host_no_idle - raw.csv'

In [265]:
#adjust header if necessary
df_data = pd.read_csv(data_folder + data_name, header = 0)

In [266]:
#remove idle vm rows

#y-cruncher
#df_data.dropna(subset=['processorInfo'], inplace= True)

In [267]:
df_data.head()

,experimentID,setId,instanceID,instanceType,wallTime,testOption,per-request-avg-time,total-time,thread-num
0,1574805970-0,0,i-08dc3567954ef1f78,c4.large,30.371954,--num-threads=2 --test=memory --memory-block-s...,1056.70ms,29.2595s,2
1,1574805970-0,0,i-08dc3567954ef1f78,c4.large,31.137912,--num-threads=2 --test=memory --memory-block-s...,1081.08ms,29.9024s,2
2,1574805970-0,0,i-0428f37fe443a7300,c4.large,36.882425,--num-threads=2 --test=memory --memory-block-s...,1287.23ms,35.6990s,2
3,1574805970-0,0,i-0428f37fe443a7300,c4.large,36.477967,--num-threads=2 --test=memory --memory-block-s...,1275.88ms,35.2930s,2
4,1574805970-0,0,i-01cc5c3fb74edb9f4,c4.large,30.276368,--num-threads=2 --test=memory --memory-block-s...,1048.67ms,29.0292s,2


In [268]:
#setup cols, metric for the relative experiment type
def setUpMetrics(experimentName):
    
    if experimentName == 'pgbench':
        cols = ['instanceID', 'setId', 'transactions']
        metric = 'transactions'
    elif experimentName == 'sysbench' or experimentName == 'y-cruncher':
        cols = ['instanceID','setId','wallTime']
        metric = 'wallTime'
    elif experimentName == 'iperf':
        df_data['Total'] = df_data['Upload'] + df_data['Download']
        cols = ['instanceID','setId','Upload', 'Download', 'Total']
        metric = 'Total'
    else:
        print('Illegal Experiment Name..')
    return metric, cols

In [269]:
#setup vars for each experiment time... i.e. wallTime vs transactions
experimentName = 'sysbench'
metric, cols = setUpMetrics(experimentName)

Illegal Experiment Name..


UnboundLocalError: local variable 'metric' referenced before assignment

In [ ]:
#create setId column
df_data['setId'] = df_data['experimentID'].str.strip().str[-2:]
df_data['setId'] = df_data['setId'].astype('int32').abs()

In [ ]:
df_data.info()

In [ ]:
df_data.tail()

In [ ]:
#drop other columns
df_data = df_data[cols]

In [ ]:
df_data.head()

In [ ]:
df_data.describe()

In [ ]:
df_data.info()

In [ ]:
#group by tenants / setId 
mean = df_data.groupby('setId').mean()
mean.head()

In [ ]:
#create result dataframe
result_df = mean

In [ ]:
#create std, normalized data
std = df_data.groupby('setId').std()
std.head()

In [ ]:
#add normalized column
if experimentName == 'pgbench':
    result_df['normalized'] = (result_df.loc[:,metric:]-result_df.iloc[-1][metric:]).div(result_df.iloc[-1][metric:])
    result_df['normalized'] *= 100
    result_df['normalized'] += 100
else: 
    result_df['normalized'] = 1 - (result_df.loc[:,metric:]-result_df.iloc[-1][metric:]).div(result_df.iloc[-1][metric:])
    result_df['normalized'] *= 100

result_df['normalized'] = result_df['normalized'].round(2).astype(str) + '%'

In [ ]:
result_df.head()

In [ ]:
#add std column
result_df['std'] = std

In [ ]:
#reverse table
result_df=result_df.iloc[::-1]

In [ ]:
#add tenants
result_df['tenants'] = np.arange(len(result_df))+1

In [ ]:
#rearrange columns
cols = ['tenants', metric, 'std', 'normalized']

In [ ]:
result_df = result_df[cols]

In [ ]:
#add % to normalized

In [ ]:
result_df

In [ ]:
#print to csv
result_df.to_csv( path_or_buf='./data/summary/' + data_name + '-Summary.csv')

In [ ]:
# graph tenants vs wallTime